# Basic homomorphic encryption
Let's start with some basic homomorphic encryption using exponentiation of a generator and modulo

The first step is to get a polynomial evaluated in the unencrypted domain, encrypt the result.
The second step is to evaluate it in the encrypted domain.
The third step is to verify whether they match, or not.

$$ p(x)= c0 + c1*x + c2*x^2 +c3*x^3 $$

The other way to represent this polynomial is by using co-factors, which uses the roots (solution) of the polynomial.

$$ p(x) = (x - r1) * (x - r2) * (x - r3) = 0 $$


In [7]:
from IPython.display import Markdown as md
import numpy as np
# Values for the user to input, if he wants to
generator = 7
field_prime = 241
#field_prime = 7919

# x^3 - 18 x^2 + 80 x - 96 = 0
# Co-factors (roots) for this polynomial are [2,4,12]

# let's define a utility function to calculate coefficients when we know the roots
def calculate_coeff_from_roots(polynomial_roots : list[int]):
    calculated_coefficients = np.polynomial.polynomial.polyfromroots(polynomial_roots)
    return list(map(int,calculated_coefficients.astype(int))) # map back to a list of int

# Let's compute the coefficents from known roots and see what we get. 
# We'll use numpy's integrated function to do this
polynomial_roots = [10,3,5]
p_polynomial_coefficients = calculate_coeff_from_roots(polynomial_roots)
target_polynomial_coefficients = []

# change_variables = input("Do you want to change variables?")
# if change_variables == "yes":
#     generator = input("generator")

equation = "$$ p(x)= "

for index, coeff in enumerate(p_polynomial_coefficients):

    if index == 0:
        equation+= str(coeff)
    else:
        if coeff >= 0:
            equation+= (" + ")
        else:
            equation+= (" - ")
        equation+= str(abs(coeff))
        equation+= (" * x^{}".format(index))
               
equation+= (" $$")
md(equation)
#md("$$ p(x)= {} + {}*x + c2*x^2 $$".format(p_polynomial_coefficients[0],p_polynomial_coefficients[1]))

$$ p(x)= -150 + 95 * x^1 - 18 * x^2 + 1 * x^3 $$

In [8]:
# define the encryption function
# all operations defined on Galois field G(p), with p being a prime number
# encryption is done by exponentiation of a generator (a prime number too) to the power of the unencrypted value
# Now fortunately, the pow() function in Python has an embedded functionality to apply a modulo
#  and it even works on large numbers, by using reduction
def encrypt(unencrypted, gen=generator,modulo=field_prime):
    encrypted = pow(gen,unencrypted,modulo)
    #print("exponentiated to : ", encrypted)
    return encrypted
    
print("encrypt 5, with generator 7 and p 19: ",encrypt(5,gen=7,modulo=19))
print("encrypt 6, with generator 7 and p 19: ",encrypt(6,gen=7,modulo=19))
print("encrypt 334499, with generator 7 and p 19: ",encrypt(334499,gen=7,modulo=19))
print('let\'s what we get with negative number -5:',encrypt(-5,gen=7,modulo=19))
print('The negative number is the same thing as applying the exponentiation to the inverse multiplicative',encrypt(14,gen=7,modulo=19) )


encrypt 5, with generator 7 and p 19:  11
encrypt 6, with generator 7 and p 19:  1
encrypt 334499, with generator 7 and p 19:  11
let's what we get with negative number -5: 7
The negative number is the same thing as applying the exponentiation to the inverse multiplicative 11


## Now we see how we can do our first encryption of polynomial
The way to do it is first to compute the polynomial in the unencrypted domain for a given r(a scalar that is used to compute the value of the polynomial, e.g. p(r)), then encrypt this result using the method described above.
Then the prover computes it in the encrypted domain, returns it to the verifier.
Finally, the Verifier verifies the 2 values match.

One of the things that has been a little complicated to integrate is how to prove equality in the encrypted domain.
So, what do we mean by encrypted domain to start with?
We mean that arithmetic operations have a different meaning than in the unencryted domain (i.e. the normal way 
we perform arithmetic)

The encrypted domain has elements that are in the form $E(x)=(g^x\mod p)$, with g and p being chosen primes.

When 2 encrypted elements are added together, we in fact use a multiplier to compute that addition. This is because if we have $E(a)=(g^a \mod p), E(b)=(g^b \mod p)$ as elements and we add them in the unencrypted domain we get $E(a+b) \equiv (g^{a+b} \mod p) \equiv (g^a * g^b \mod p) \equiv (g^a \mod p) * (g^b \mod p) \equiv (E(a) * E(b))$

For instance, in the unencrypted domain $1 + 5 = 6$, if we want to compute it in the encrypted domain with $g=3, p=7$. 
We start by computing 6 in the encrypted domain: $E(6) = 3^6 \mod 7 = 1$
Then we compute $E(1) * E(5) = (3^5 \mod 7) * (3^1 \mod 7) = 5 * 3 \mod 7 = 1$

Now when we multiply 2 encrypted elements what we really are doing is exponentiating them. This is because again we are dealing with exponentiated elements. So let's see what we get when we perform a multiplication in the encrypted domain. $E(a * b) = (g^{a*b} \mod p) = ((g^a)^b \mod p)$. Rightly, you may be wondering if the multiplication in the encrypted domain is commutative, if in fact $((g^a)^b \mod p) = ((g^b)^a \mod p)$ ?

Let's take an example, with a=4 and b=5, and again g=3, p=7. In the unencrypted domain $5 * 4 = 20$, if we then encrypt this value we get $3^{20} \mod 7 = 2$.

Now to the encrypted domain:

The first equation to check is $((g^a)^b \mod p)$. So this is $(3^4)^5 \mod 7 \equiv (3^4 \mod 7)^ 5 \mod 7) \equiv 4^5 \mod 7 = 2$ ! success
The second equation to check is $((g^b)^a \mod p)$. So this is $(3^5)^4 \mod 7 \equiv (3^5 \mod 7)^ 4 \mod 7) \equiv 5^4 \mod 7 = 2$ ! success again

The 2 things that could have made this not true is the fact we use modulo at each step combined with _the power of a power rule_.

A negative number in the encrypted domain is found by calculating the multiplicative modular inverse of that number in the modulo field and is represented by $g^{-1} \mod p$, which means $g * g^{-1} = 1 (\mod p )$. And as we have seen above with the power rule of a power rule, we can rewrite $g^{-2} \mod p$ as $(g^{-1})^2 (\mod p )$

For instance if we want to encrypt -2, we get 4 by virtue of $3^{-2} \mod 7 \equiv (3^{-1})^2) \mod7$. The modular multiplication inverse of 3 is 5 (because $3 * 5 mod 7 = 1$). So we can rewrite that last equation with $5^2 \mod 7 = 4$. Case solved!


When we perform a division in the encryted domain, we have to use modulo division concept which is also based on the modular inverse but I will not explain it, because we it is not needed.

Finally, it is NOT possible have the equivalent of exponentiation of the unencrypted domain in the encrypted one.

In [9]:
# We have defined the polynomial earlier with its coefficients in p_polynomial_coefficients
# let's compute the polynomial p(r) value for a given r (not a root)
r=20
evaluated_poly = np.polynomial.polynomial.polyval(r, p_polynomial_coefficients)
print('the polynomial evaluated at r: ',evaluated_poly)
# Now let's encrypt this value and see what we get
encrypted_result = encrypt(int(evaluated_poly))
print('encrypted result: ',encrypted_result)

# The second part is to verify that we can get the same result when doing the computation in the encryted domain
# How we do this is by having the verifier (he chooses a secret value where to evaluate the polynomial) give the encrypted value
# of each x^i for the value r. In our case the prover needs the encrypted values for 11^3, 11^2, 11^1 and 11^0
# For now we just want to see it in action, we are not yet implementing the protocol for Zero Knowledge proof
encrypted_powers = []
for index, order in enumerate(p_polynomial_coefficients):
    encrypted_powers.append(encrypt(pow(r,index)))

# Now let's compute E (p(11)) = encrypted_powers[0]^coeff[0] * encrypted_powers[1]^coeff[1] * encrypted_powers[2]^coeff[2] * encrypted_powers[3]^coeff[3] 
# We apply modulo a few times along the way to reduce the chance of overflow
result = 1
for index,coeff in enumerate(p_polynomial_coefficients):
    result = (result * pow(encrypted_powers[index],int(coeff),field_prime)) % field_prime
print('the result is: ....',result)


the polynomial evaluated at r:  2550.0
encrypted result:  211
the result is: .... 211


In [10]:

# I think from here on out it would be good to have a verifier and a prover classes and clearly delineate the tasks they each
# must perform. As the functionality will evolve as we make the protocol more complex, the classes will be named 
# using the chapter number defining the protocol

# We will need random values from here on out, let's use the secrets module for it
import secrets

# There are some limitations with using the numpy package to evaluate the polynomial when using large numbers
# Because of this we will use a [package](https://mhostetter.github.io/galois/latest/tutorials/intro-to-prime-fields/#prime-field) that can do polynomial evaluation in a Galois Field
import galois

# A few things are different from the example given in the preceding section
# The verifier knows a polynomial t(x), called the target polynomial
# The prover claims he knows a polynomial of order n p(x) that is divisible by t(x) (thus has t(x)'s roots as co-factors)
# The prover wants to keep p(x) secret, but wants to prove he knows it
# Thus the prover computes p(x)/t(x) = h(x), and will provide the value of p(s) and h(s) to the verifier
# The prover uses s that is provided by the verifier, and the computation occurs in the encrypted domain
# As is mentioned in the pdf document, we cannot exponentiate in the encrypted domain
# That is why the verifier must provide the encrypted values of s for all orders of the polynomial
# That is E(s^0), E(s^1), ... , E(s^n), where n is the polynomial order
# Finally once the prover returns p(s) and h(s) to the verifier, the latter computes p(s)/t(s) and if it 
# matches p(s), then he knows it's correct (very bad protocol for now, but it'll get more secure)

# Because we are going to implement multiple version of the protocol, I'll use the protocol module
# There's a good explanation how it works at: https://idego-group.com/blog/2023/02/21/we-need-to-talk-about-protocols-in-python/
from typing import Protocol

class Verifier(Protocol):
    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly: int, field_prime:int = field_prime, generator:int = generator):
        super().__init__()
        self.encrypted_powers = []
        self.field_prime = field_prime
        self.generator = generator
        self.target_polynomial_coefficients = target_polynomial_coefficients
        self.target_desc_coeffs = list(reversed(target_polynomial_coefficients))
        self.s : int
        self.max_order_p_poly = max_order_p_poly
    
    def calculate_encrypted_powers(self):
        ...
    
    def evaluate_unencrypted_target_polynomial(self):
        ...
    
    def set_secret(self, s:int):
        ...
    

class Verifier_334(Verifier):
# The verifier's tasks are as follows:
# (a) sample a random value s, i.e., secret
# (b) calculate encryptions of s for all powers i in 0, 1, ..., d, i.e.: E(si) = g^s^i
# (c) evaluate unencrypted target polynomial with s: t(s)
# (d) encrypted powers of s are provided to the prover: E(s0), E(s1), ..., E(sd)
# (e) The last step for the verifier is to checks that p = t(s) · h in encrypted space: 
# g^p =(g^h)^t(s) ⇒ g^p = g^t(s)·h

    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly:int, field_prime:int = field_prime, generator:int = generator):
        super().__init__(target_polynomial_coefficients, max_order_p_poly, field_prime, generator)
        self.s = secrets.randbelow(self.field_prime) # (a)
        print(f'secret = {self.s}')
    
    def calculate_encrypted_powers(self):
        # (b) & (d)
        for index in range(self.max_order_p_poly):
            self.encrypted_powers.append(encrypt(pow(self.s,index)))

    def evaluate_unencrypted_target_polynomial(self):
        self.evaluated_poly = int(np.polynomial.polynomial.polyval(self.s, self.target_polynomial_coefficients))
        print(f'the polynomial evaluated at s = {self.s} is {evaluated_poly} ')

    def set_secret(self, s:int):
        self.s = s

    def check_encrypted_t_h_p(self, encrypted_h:int, encrypted_p:int) -> bool:
        # (e) 
        return (pow(encrypted_h,self.evaluated_poly,self.field_prime) == encrypted_p)


# let's exercise this new class
# target poly is (x-1)(x-2), which is equivalent to 2 -3x +x^2
# myVerifier =  Verifier_334([2,-3,1],max_order_p_poly=4)
# print(myVerifier.target_polynomial_coefficients)
# myVerifier.evaluate_unencrypted_target_polynomial()
# myVerifier.calculate_encrypted_powers()

# # just to make sure everything works fine, let's test this implementation with the same values we used in the 
# # preceding block
# myValidatedVerifier = Verifier_334(p_polynomial_coefficients, max_order_p_poly=4)
# myValidatedVerifier.set_secret(20)
# print(myValidatedVerifier.target_polynomial_coefficients)
# myValidatedVerifier.evaluate_unencrypted_target_polynomial()

# it is not working, the reason is that any value used in the computation of the GF must be in the finite field.
# so what we see is that the evaluation of the polynomial in the first implementation returns 2550 for r=20
# With the GF, the evaluated value is 140, which is 2550 mod 241! For this test we have been using 241 as the field prime
# How can we explain that the prover's result matches the verifier's then? 
# it seems to me that there is a good chance that is because the power of r values are outside of the field limit
# So, if we take a field prime that is big enough, the results will match. Let's see if this works: new field prime = 7919
# Result is that it works indeed! But I think the field prime modulo should be used on the powers of r too.
# This seems to give different results when reverting back to 241 as field prime... mystery to be solved 
# when implementing the prover. 
# Ok, so after thinking about this more, here is the resolution for this problem. Whenever computations occur in the
# non-encrypted domain, one must NOT apply the field restiction (modulo) until we go to the encrypted domain.
# So what this means is that when we evaluate the polynomial at r, we must do it in the unencrypted domain and THEN
# encrypt that value. Also, when we generate the powers of r, we must do it in then unencrypted domain and THEN encrypt them.


In [14]:

# This is where the prover class is implemented and it will be using Protocol and named
# using the chapter number defining the protocol in the PDF
import numpy as np

class Prover(Protocol):
    def __init__(self, powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__()
        self.field_prime = field_prime
        self.GF = galois.GF(field_prime)
        self.encrypted_powers = powers_of_s
        self.generator = generator
        self.target_polynomial_coefficients = target_polynomial_coefficients
        self.p_polynomial_coefficients = p_polynomial_coefficients
        self.target_desc_coeffs = list(reversed(target_polynomial_coefficients))
    
    def calculate_h(self):
        ...
    def evaluate_encrypted_p(self):
        ...
    def evaluate_encrypted_h(self):
        ...
    

class Prover_334(Prover):
# The prover's tasks are as follows:
# (a) Calculate polynomial h(x) = p(x)/t(x) 
# (b) Using encrypted powers g^s^0, g^s^1, ... , g^s^d and coefficients c0, c1, ... , cn 
# evaluates E (p(s)) = g^p(s) = (g^s^d)^cd ··· (g^s^1)^c1 · ( g^s^0 )^c0 
# (c) and similarly E (h(s)) = g^h(s) 
# (d) The resulting g^p and g^h are provided to the verifier


    def __init__(self, powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__(powers_of_s, target_polynomial_coefficients, p_polynomial_coefficients, field_prime, generator)

    
    def calculate_h(self):
        # (a)
        calculated_h = np.polynomial.polynomial.polydiv(self.p_polynomial_coefficients,self.target_polynomial_coefficients)
        if calculated_h[1] != 0:
            raise ValueError('polynomial is not divisible')
        else:
            self.h_polynomial_coefficients = list(map(int, calculated_h[0].astype(int)))
        print(f'h polynomial is {self.h_polynomial_coefficients}')

    def evaluate_encrypted_p(self):
        #print(galois.primitive_root(self.field_prime))
        # so what I need to do is array multiplication between encrypted powers and p_polynomial_coefficients
        # in the GF. Is that possible? Well it is, but it returns non modulo values, which is a problem.
        result = 1
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            result = (result * pow(self.encrypted_powers[index],int(coeff),field_prime)) % field_prime
        self.encrypted_p = result


    def evaluate_encrypted_h(self):
        # do the same thing as with p(x), but shrink encrypted powers to right size
        result = 1
        for index, coeff in enumerate(self.h_polynomial_coefficients):
            result = (result * pow(self.encrypted_powers[index],int(coeff),field_prime)) % field_prime
        self.encrypted_h = result
# let's exercise this new class
# target poly is (x-1)(x-2), which is equivalent to 2 -3x +x^2
# we now want to see what happens with p(x) = (x-1)(x-2)(x-3), which is x^3 - 6 x^2 + 11 x - 6
# print(f'Encrypted powers are: {myVerifier.encrypted_powers}')
# myProver =  Prover_334(myVerifier.encrypted_powers, [2,-3,1],[-6,11,-6,1])
# print(f'p(x) = {myProver.p_polynomial_coefficients}')
# myProver.calculate_h()
# myProver.evaluate_encrypted_p()
# myProver.evaluate_encrypted_h()


# print(f'The E(P(s)) is {myProver.encrypted_p}, my E(H(s)) is {myProver.encrypted_h}')
# encrypted_p = myProver.encrypted_p
# encrypted_h = myProver.encrypted_h
# print(f'T(s) = {myVerifier.evaluated_poly}')
# print(f'the answer is : {pow(encrypted_h,myVerifier.evaluated_poly,field_prime)}')

In [20]:
# Now let's use the verifier and prover classes with another bigger sized example
# The P(x) will be (x - 1) (x - 2) (x - 3) (x - 4)
# The target polynomial T(x) will be (x - 1) (x - 2) (x - 3)
# The P(x)/T(x) = H(x) will be (x-4)

t_poly_coeffs = calculate_coeff_from_roots([1,2,3])
p_poly_coeffs = calculate_coeff_from_roots([1,2,3,4])
myVerifier = Verifier_334(t_poly_coeffs,5)
myVerifier.calculate_encrypted_powers()
myVerifier.evaluate_unencrypted_target_polynomial()
myProver = Prover_334(myVerifier.encrypted_powers,t_poly_coeffs,p_poly_coeffs)
myProver.calculate_h()
myProver.evaluate_encrypted_h()
myProver.evaluate_encrypted_p()
if myVerifier.check_encrypted_t_h_p(myProver.encrypted_h,myProver.encrypted_p):
    print(f'Everything checks out')

secret = 213
the polynomial evaluated at s = 213 is 2550.0 
h polynomial is [-4, 1]
Everything checks out


The next step is to restrict the usage of the polynomial.
What do we mean by this?

Well, we can see that the protocol implemented from chapter 3.3.4 has a major weakness, which is that the prover does NOT have to use the encrypted powers, or even a polynomial to provide a valid answer.
One could use any possible means to find some arbitrary values $z^p$ and $z^h$ which satisfy equation $z^p = (zh)^{t(s)}$ and provide them to the verifier instead of $g^p$ and $g^h$. For example, for some random r, $zh = g^r$ and $zp = (g^{t(s)})^r$, where $gt(s)$ can be computed from the provided encrypted powers of s. That is why the verifier needs the proof that only supplied encryptions of powers of s were used to calculate gp and gh and nothing else.

So, we are going to force the prover to use the encrypted powers by having him perform the same calculations twice, but with encrypted powers $\alpha$ shifted. Let's explore this concept to understand it and see how it applies here.

Alice chooses a random value $\alpha$, which she uses to exponentiate a value x that she requires Bob to exponentiate. So, she generates $\alpha$ (let's say 3) and wants x (let's say it's 2) to be exponentiated. She generates $x' = x^{\alpha}$ and gives x and x' to Bob. He does not know what $\alpha$ is. For this example, she gives Bob 2 and $2^3 \mod 7$

Now Bob exponentiate x and x' to the power of z = 6 (his choice) in the prime field p=7. So computes $y = x^z \mod p$ and $y' = x'^z \mod p$. If 